In [1]:
import findspark
findspark.init()

In [2]:
#Importamos spark
from pyspark.sql import SparkSession

In [3]:
#creamos la sesion en spark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ETL Project") \
    .getOrCreate()


In [4]:
spark

In [7]:
import pandas as pd

# Leer el archivo CSV usando pandas
csv_data = pd.read_csv('C:/US_Accidents_March23_corrected.csv')

# Mostrar las primeras 10,000 filas
print(csv_data.head(10000))


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_24348\3345494255.py:4: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_data = pd.read_csv('C:/US_Accidents_March23_corrected.csv')


             ID   Source  Severity                     Start_Time  \
0     A-2047758  Source2       2.0            2019-06-12 10:10:56   
1     A-4694324  Source1       2.0  2022-12-03 23:37:14.000000000   
2     A-5006183  Source1       2.0  2022-08-20 13:13:00.000000000   
3     A-4237356  Source1       2.0            2022-02-21 17:43:04   
4     A-6690583  Source1       2.0            2020-12-04 01:46:00   
...         ...      ...       ...                            ...   
9995  A-5038936  Source1       2.0            2022-07-27 14:32:54   
9996  A-4738290  Source1       2.0            2022-02-08 16:43:40   
9997  A-1408429  Source2       2.0            2020-08-01 08:54:01   
9998  A-6532324  Source1       2.0            2021-01-07 23:42:07   
9999  A-4706401  Source1       2.0            2022-12-30 00:01:00   

                           End_Time  Start_Lat   Start_Lng    End_Lat  \
0               2019-06-12 10:55:58  30.641211  -91.153481        NaN   
1     2022-12-04 01:56:53

In [21]:
import pandas as pd

# Leer el archivo CSV usando pandas
csv_data = pd.read_csv('C:/US_Accidents_March23_corrected.csv')

# Columnas a eliminar
columns_to_drop = ['ID', 'Source', 'Country', 'Description', 'End_Lat', 'End_Lng', 
                'Civil_Twilight', 'Nautical_Twilight', 'Astronomical_Twilight;;;;;;;;;;']

# Eliminación de las columnas
df_cleaned = csv_data.drop(columns=columns_to_drop)

# Imputar valores nulos en columnas numéricas con la media
df_cleaned['Temperature(F)'].fillna(df_cleaned['Temperature(F)'].mean(), inplace=True)

# Imputar valores nulos en columnas categóricas con la moda
df_cleaned['Weather_Condition'].fillna(df_cleaned['Weather_Condition'].mode()[0], inplace=True)

# Imputar columnas numéricas con la media
num_cols = ['Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)', 'Precipitation(in)']
df_cleaned[num_cols] = df_cleaned[num_cols].apply(lambda col: col.fillna(col.mean()))

# Imputar columna categórica con el valor más frecuente
df_cleaned['Wind_Direction'] = df_cleaned['Wind_Direction'].fillna(df_cleaned['Wind_Direction'].mode()[0])

# Imputar Weather_Timestamp con el valor más cercano (por tiempo)
df_cleaned['Weather_Timestamp'] = df_cleaned['Weather_Timestamp'].fillna(method='ffill')

# Eliminar filas que contienen valores nulos
df_cleaned.dropna(inplace=True)

# Contar valores nulos (NaN) en cada columna
nan_counts = df_cleaned.isna().sum()

# Contar valores vacíos ('') en cada columna
empty_counts = (df_cleaned == '').sum()

# Combinar los conteos en un solo DataFrame para visualizar mejor
null_summary = pd.DataFrame({
    'NaN Count': nan_counts,
    'Empty String Count': empty_counts,
    'Total Missing': nan_counts + empty_counts
})

# Mostrar el resumen
print(null_summary)

# Configuración para mostrar más filas y columnas si es necesario
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', None)

# Mostrar el DataFrame en formato tabular
df_cleaned.head(100)


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_24348\2826111046.py:4: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_data = pd.read_csv('C:/US_Accidents_March23_corrected.csv')
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_24348\2826111046.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Temperature(F)'].fillna(df_cleaned['Temperature(F)'].mean(), inplace=True)
C:\Users\USUARIO\AppData\Local\Temp\ipykernel_24348\28261

                   NaN Count  Empty String Count  Total Missing
Severity                   0                   0              0
Start_Time                 0                   0              0
End_Time                   0                   0              0
Start_Lat                  0                   0              0
Start_Lng                  0                   0              0
Distance(mi)               0                   0              0
Street                     0                   0              0
City                       0                   0              0
County                     0                   0              0
State                      0                   0              0
Zipcode                    0                   0              0
Timezone                   0                   0              0
Airport_Code               0                   0              0
Weather_Timestamp          0                   0              0
Temperature(F)             0            

,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Zipcode,Timezone,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset
0,2.0,2019-06-12 10:10:56,2019-06-12 10:55:58,30.641211,-91.153481,0.000,Highway 19,Zachary,East Baton Rouge,LA,70791-4610,US/Central,KBTR,2019-06-12 09:53:00,77.000000,77.000000,62.000000,29.920000,10.000000,NW,5.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
1,2.0,2022-12-03 23:37:14.000000000,2022-12-04 01:56:53.000000000,38.990562,-77.399070,0.056,Forest Ridge Dr,Sterling,Loudoun,VA,20164-2813,US/Eastern,KIAD,2022-12-03 23:52:00,45.000000,43.000000,48.000000,29.910000,10.000000,W,5.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
2,2.0,2022-08-20 13:13:00.000000000,2022-08-20 15:22:45.000000000,34.661189,-120.492822,0.022,Floradale Ave,Lompoc,Santa Barbara,CA,93436,US/Pacific,KLPC,2022-08-20 12:56:00,68.000000,68.000000,73.000000,29.790000,10.000000,W,13.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
3,2.0,2022-02-21 17:43:04,2022-02-21 19:43:23,43.680592,-92.993317,1.054,14th St NW,Austin,Mower,MN,55912,US/Central,KAUM,2022-02-21 17:35:00,27.000000,15.000000,86.000000,28.490000,10.000000,ENE,15.000000,0.000000,Wintry Mix,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
4,2.0,2020-12-04 01:46:00,2020-12-04 04:13:09,35.395484,-118.985176,0.046,River Blvd,Bakersfield,Kern,CA,93305-2649,US/Pacific,KBFL,2020-12-04 01:54:00,42.000000,42.000000,34.000000,29.770000,10.000000,CALM,0.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
5,2.0,2021-03-29 07:03:58,2021-03-29 08:51:01,42.532082,-70.944267,0.000,Lowell St,Peabody,Essex,MA,01960-4275,US/Eastern,KBVY,2021-03-29 06:53:00,42.000000,35.000000,58.000000,29.370000,10.000000,W,13.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,True,False,Day
6,2.0,2020-01-14 16:49:23,2020-01-14 20:49:23,42.421280,-123.119450,0.000,I-5 N,Gold Hill,Jackson,OR,97525,US/Pacific,KMFR,2020-01-14 16:53:00,35.000000,35.000000,89.000000,28.710000,10.000000,CALM,0.000000,0.000000,Light Rain,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
7,2.0,2021-08-13 16:48:00.000000000,2021-08-13 19:09:09.000000000,30.191010,-85.682508,0.047,Claremont Dr,Panama City,Bay,FL,32405-3534,US/Central,KPAM,2021-08-13 16:56:00,90.000000,90.000000,55.000000,30.030000,10.000000,SW,12.000000,0.000000,Fair,False,False,False,False,False,False,False,False,False,False,False,False,False,Day
8,2.0,2022-10-12 13:59:30,2022-10-12 15:33:53,32.868947,-96.804018,0.038,Preston Rd,Dallas,Dallas,TX,75225,US/Central,KDAL,2022-10-12 13:53:00,91.000000,91.000000,39.000000,29.340000,10.000000,VAR,7.000000,0.000000,Fair,False,False,False,False,False,False,False,False,True,False,False,False,False,Day
9,2.0,2021-10-21 07:39:30,2021-10-21 09:24:30,39.717218,-86.124691,1.301,I-65,Indianapolis,Marion,IN,46237,US/Eastern,KIND,2021-10-21 07:47:00,63.000000,63.000000,78.000000,29.060000,10.000000,SW,10.000000,0.000000,Cloudy,False,False,False,False,False,False,False,False,False,False,False,False,False,Night
